# [문제]
-low_2.docx 파일을 읽고, Chroma 저장
-LLM 질문 -> 답변
-전세사기 피해에 관한 법률 질문만 받기
-이 외의 질문은 '답변을 할 수 없습니다.'

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
docs = Docx2txtLoader("law_2.docx").load()
print(docs[0].page_content)  # 첫 문서 내용 출력



전세사기피해자 지원 및 주거안정에 관한 특별법 시행령 ( 약칭: 전세사기피해자법 시행령 )

[시행 2024. 11. 11.] [대통령령 제34987호, 2024. 11. 8., 일부개정]



국토교통부(피해지원총괄과) 044-201-5233, 5234



제1조(목적) 이 영은 「전세사기피해자 지원 및 주거안정에 관한 특별법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.

제2조(전세사기피해지원단) ① 「전세사기피해자 지원 및 주거안정에 관한 특별법」(이하 “법”이라 한다) 제10조제1항에 따른 전세사기피해지원단(이하 “지원단”이라 한다)은 단장 1명과 단원으로 구성한다.

  ② 지원단의 단장은 국토교통부의 고위공무원단에 속하는 일반직공무원 중에서 국토교통부장관이 지명하는 사람이 된다.

  ③ 지원단의 단장은 국토교통부장관의 명을 받아 지원단의 업무를 총괄하고, 지원단의 단원을 지휘ㆍ감독한다.

  ④ 제1항부터 제3항까지에서 규정한 사항 외에 지원단의 구성 및 운영 등에 필요한 사항은 국토교통부장관이 정한다.

제3조(국세의 안분 방법 및 신청 등) ① 법 제23조제1항 각 호 외의 부분 전단에 따라 안분하여 징수하는 전세사기피해주택 임대인의 국세는 다음 각 호의 구분에 따라 산정한다.

  1. 상속세, 증여세 및 종합부동산세의 경우: 고지 또는 신고 건별로 각각 가목에 따라 계산한 금액과 나목의 금액 중 큰 금액

    가. 다음의 계산식에 따라 계산한 금액

  

    나. 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액 중 「국세기본법」 제35조제3항에 따른 해당 재산에 대하여 부과된 국세 금액

  2. 상속세, 증여세 및 종합부동산세 외의 국세의 경우: 고지 또는 신고 건별로 각각 제1호가목에 따라 계산한 금액

  ② 제1항제1호가목의 계산식에서 “전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합계액”은 법 제23조제3항 전단에 따라 안분 적용 신청을 한 날이 속하는 연도의 직전 연도의

In [3]:
pip install langchain-community langchain-openai chromadb python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from dotenv import load_dotenv

# 환경 변수 로드 (OPENAI_API_KEY 필요)
load_dotenv()

# 문서 로드
loader = Docx2txtLoader("law_2.docx")
documents = loader.load()

# 임베딩 모델 설정
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

# Chroma 벡터 DB에 저장
db = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory="./chroma",
    collection_name="chroma-law"
)

# LLM 모델 설정
llm = ChatOpenAI(model="gpt-4o")

# 질의
query = "전세사기 피해자에 해당하는 기준은 무엇인가요?"

# 유사 문서 검색
similar_docs = db.similarity_search(query=query, k=3)

# 프롬프트 구성
prompt_template = """
당신은 전세사기 피해 법률 전문가입니다.
아래 문서를 참고하여 사용자의 질문에 답변해주세요.
전세사기 피해에 관한 법률 질문만 받습니다. 그 외의 질문은 '답변을 할 수 없습니다.'라고 답해주세요.

[참고 문서]
{retrieved_docs}

[질문]
{query}
"""

formatted_prompt = prompt_template.format(
    retrieved_docs="\n\n".join([doc.page_content for doc in similar_docs]),
    query=query
)

# LLM에 프롬프트 전달
response = llm.invoke(formatted_prompt)

response.content


'전세사기 피해자는 다음의 기준을 모두 충족해야 합니다. \n\n1. 임차인이 주택을 인도받고 주민등록을 마치며, 임대차계약증서상의 확정일자를 갖출 것(임차권등기를 마친 경우 또는 전세권이 설정된 경우도 포함).\n   \n2. 임차보증금이 5억원 이하일 것. 다만, 전세사기피해지원위원회가 시ㆍ도별 여건 등을 고려하여 상향 조정할 수 있습니다.\n\n3. 임대인의 파산, 임차주택의 경매나 공매 절차 개시 등으로 인해 2인 이상의 임차인이 임차보증금 반환을 받지 못하는 피해가 발생했거나 예상될 것.\n\n4. 임대인에 대한 수사가 개시되었거나, 임대인이 임차보증금 반환 능력이 없다는 등의 이유가 있어 임차보증금 반환채무를 이행하지 않을 의도가 있었다고 의심할 만한 상당한 이유가 있을 것.\n\n단, 다음 경우는 적용 대상에서 제외됩니다.\n\n- 임차인이 보증금 반환을 위한 보증 또는 보험에 가입했거나 임대인이 보증에 가입하여 임차인에게 보증금의 전액 반환이 가능한 경우\n- 임차인의 보증금 전액이 주택임대차보호법에 따른 최우선변제가 가능한 경우\n- 임차인이 대항력 또는 우선변제권 행사를 통해 보증금 전액을 자력으로 회수할 수 있는 경우\n\n이러한 절차들을 거쳐 전세사기 피해자로 지원받을 수 있습니다.'

In [5]:
query = "오늘 날씨 어때?"
